In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [2]:
# dataset = pd.read_csv('validation_data_prediction.csv')
dataset = pd.read_csv('testing_data_prediction.csv')
leverage_factor = 10
increase_factor = 1.5

In [3]:
dataset["new_rfr"] = dataset["daily_risk_free_rate"] * increase_factor

In [4]:
def first_trading_strategy(predictions: pd.DataFrame,
                           initial_cash: float = 200000,
                           leverage_factor: float = leverage_factor):
    
    # initialize the variables
    cash = [initial_cash*leverage_factor]
    margin = [initial_cash]
    position = [0]
    re = [0]
    bank_account = [0]
    high_bank_account = [0]
    previous_price = float('nan')
    long_price = short_price = 0
    price = []
    
    # iterate through the predictions
    for _, row in predictions[:-1].iterrows():
        current_date = row['Date']
        current_price = row['Close']
        
        current_rfr = row['daily_risk_free_rate']
        current_high_rfr = row['new_rfr']
        
        tomorrow_prediction = row['Tomorrow Prediction']
        
        current_cash = cash[-1]
        current_margin = margin[-1]
        current_position = position[-1]
        current_re = re[-1]
        current_bank_account = bank_account[-1]
        currnet_high_bank_account = high_bank_account[-1]
        price.append(current_price)
        
        # checking the excess returns
        # excess return happens when the price goes down and we have a short position
        # or the price goes up and we have a long position
        if current_position * current_price > current_margin * leverage_factor:
            excess_return = current_position * current_price - current_margin * leverage_factor
            excess_stock = excess_return / current_price
            current_position -= excess_stock
            current_bank_account += excess_return
            
        if -current_position * (current_price) > current_margin * leverage_factor:
            excess_return = (-current_position * current_price) - current_margin * leverage_factor
            excess_stock = excess_return / current_price
            current_position += excess_stock
        
        # check how much money we lost on wrong predictions
        if current_position > 0 and current_price < long_price:
            unit_loss = long_price - current_price
            loss = unit_loss * stock_for_long
            current_re = unit_loss / long_price
            if current_margin - loss < 0:
                print(f"Game Over! You lost all your money.")
                break
        
        if current_position < 0 and current_price > short_price:
            unit_loss = current_price - short_price
            loss = unit_loss * stock_for_short
            current_re = unit_loss / short_price
            if current_margin - loss < 0:
                print(f"Game Over! You lost all your money.")
                break
        
        # check if we should open a position
        # if we think the price will go up, we will close the short position and 
        # open a long position
        if tomorrow_prediction > current_price:
            # if we have a short position, we will close it
            if current_position < 0:
                stock_for_short = abs(current_position)
                unit_profits = short_price - current_price
                profits = unit_profits * stock_for_short
                current_position = 0
                current_margin += profits
                current_re = unit_profits / short_price
                if current_margin < 0:
                    print(f"Game Over! You lost all your money.")
                    break
                
            # don't open a long position if we already have one
            if current_position == 0:
                current_cash = current_margin * leverage_factor
                stock_for_long = current_cash / current_price
                current_position += stock_for_long
                long_price = current_price
                
        # check if we should close a position
        # if we think the price will go down, we will close the long position 
        # but not open a short position
        elif tomorrow_prediction < current_price:
            # if we have a long position, we will close it
            if current_position > 0:
                stock_for_long = current_position
                unit_profits = current_price - long_price
                profits = unit_profits * stock_for_long
                current_position = 0
                current_margin += profits
                current_re = unit_profits / long_price
                if current_margin < 0:
                    print(f"Game Over! You lost all your money.")
                    break
                    
            if current_position == 0:
                current_cash = current_margin * leverage_factor                
                stock_for_short = current_cash / current_price
                current_position -= stock_for_short
                short_price = current_price
        
        currnet_high_bank_account = current_bank_account
        
        current_interest = current_bank_account * current_rfr
        current_bank_account += current_interest
        
        current_high_interest = currnet_high_bank_account * current_high_rfr
        currnet_high_bank_account += current_high_interest
        
        # update the variables
        cash.append(current_cash)
        margin.append(current_margin)
        position.append(current_position)
        re.append(current_re)
        bank_account.append(current_bank_account)
        high_bank_account.append(currnet_high_bank_account)

    current_price = predictions.iloc[-1]['Close']
    current_date = predictions.iloc[-1]['Date']
    current_cash = cash[-1]
    current_margin = margin[-1]
    current_position = position[-1]
    current_re = re[-1]
    current_bank_account = bank_account[-1]
    currnet_high_bank_account = high_bank_account[-1]
    
    current_interest = current_bank_account * current_rfr
    current_bank_account += current_interest
    
    current_high_interest = currnet_high_bank_account * current_high_rfr
    currnet_high_bank_account += current_high_interest
    
    if current_position > 0:
        stock_for_long = current_position
        unit_profits = current_price - long_price
        profits = unit_profits * stock_for_long
        current_position = 0
        current_margin += profits
        current_re = unit_profits / long_price
        
    if current_position < 0:
        stock_for_short = abs(current_position)
        unit_profits = short_price - current_price
        profits = unit_profits * stock_for_short
        current_position = 0
        current_margin += profits
        current_re = unit_profits / short_price
    
    if current_margin < 0:
        print(f"Game Over! You lost all your money.")
    
    cash.append(current_cash)
    margin.append(current_margin)
    position.append(current_position)
    re.append(current_re)
    bank_account.append(current_bank_account)
    price.append(current_price)
    high_bank_account.append(currnet_high_bank_account)
    
    return cash, margin, position, re, bank_account, price, high_bank_account
    
cash, margin, position, re, bank_account, price, high_bank_account = first_trading_strategy(dataset)

In [5]:
dataset.index = pd.to_datetime(dataset['Date'])
dataset.drop(columns=['Date'], inplace=True)

In [6]:
report_df = pd.DataFrame({
    'margin': margin[1:],
    'position': position[1:],
    'cash': cash[1:],
    "bank_account": bank_account[1:],
    "bank_account_with_high_rfr": high_bank_account[1:],
    'excess_return': re[1:],
})
report_df.index = dataset.index

In [7]:
target_data = dataset.copy()
target_data = target_data.drop(columns=['Open', 'High', 'Low', 'Volume', 'Adj Close', "excess return", "return", "Tomorrow Prediction"])
final_result = pd.concat([target_data, report_df], axis=1)

In [8]:
final_result["theta"] = final_result["position"] * final_result["Close"]
final_result["pnl"] = (final_result["excess_return"] - final_result["daily_risk_free_rate"]) * final_result["theta"]

final_result["pnl_capital"] = final_result["bank_account"] - final_result["bank_account"].shift(1)

final_result["pnl_total"] = final_result["pnl"] + final_result["pnl_capital"]

final_result["cumulative_pnl"] = final_result["pnl"].cumsum()

final_result["cumulative_pnl_capital"] = final_result["pnl_capital"].cumsum()

final_result["cumulative_pnl_total"] = final_result["pnl_total"].cumsum()
final_result["turn_over_theta"] = abs(final_result["theta"] - final_result["theta"].shift(1))
final_result["turn_over_theta"] = final_result["turn_over_theta"].fillna(0)
final_result["cumulative_turn_over_theta"] = final_result["turn_over_theta"].cumsum()
final_result["turn_over_units"] = abs(final_result["position"] - final_result["position"].shift(1))
final_result["turn_over_units"] = final_result["turn_over_units"].fillna(0)
final_result["Upper_bound"] = final_result["margin"] * leverage_factor
final_result["Lower_bound"] = -final_result["margin"] * leverage_factor

final_result["pnl_150"] = (final_result["excess_return"] - final_result["new_rfr"]) * final_result["theta"]
final_result["pnl_capital_150"] = final_result["bank_account_with_high_rfr"] - final_result["bank_account_with_high_rfr"].shift(1)
final_result["pnl_capital_150"] = final_result["pnl_capital_150"].fillna(0)
final_result["pnl_total_150"] = final_result["pnl_150"] + final_result["pnl_capital_150"]
final_result["cumulative_pnl_150"] = final_result["pnl_150"].cumsum()
final_result["cumulative_pnl_capital_150"] = final_result["pnl_capital_150"].cumsum()
final_result["cumulative_pnl_total_150"] = final_result["pnl_total_150"].cumsum()

In [9]:
final_result["bank_account"][-1] + final_result["margin"][-1]

/var/folders/20/0y46cgg97gs8kf_s9bthlzgm0000gn/T/ipykernel_75759/1483742896.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  final_result["bank_account"][-1] + final_result["margin"][-1]


1287652.1556723977

In [10]:
# final_result.to_csv("./validation/ARIMA_validation.csv")
# final_result.to_csv("./testing/ARIMA_testing.csv")